In [ ]:
from utils import MusicFingerPrint, MusicFingerPrintFromData
from utils import invertedIndexTable, queryFromTable, musicToFolderMatching, folderToFolderMatching
import time, pickle

## Example Usages

### 1.  Build inverted index for the musics in a folder

In [ ]:
# Use multithread, the params after the first line can be omitted, in that case, you use the default values
inverted_index = invertedIndexTable('Data/GTZAN/', multithread=True, num_workers=4,
                                       window_length_ms=100, hop_length_ms=20, zero_padding=4,
                                       delta_T_ms=200, n_bands=20, target_zone_time_s=0.5, target_zone_time_offset_s=0.1,
                                       target_zone_freq_factor=0.5, progress_bar=True, CFAR_flag=True) 

In [ ]:
# Not use multithread, omit the params after the first line
inverted_index = invertedIndexTable('Data/GTZAN/', multithread=False) 

### 2.  Save/Load the calculated inverted-index table if necessary

In [ ]:
# Save
path = 'Inverted_Index/GTZAN_STFT_inverted_index_table.pkl'
confirmation = input(f"Are you sure you want to save the inverted index table to {path}? (y/n): ")
if confirmation.lower() == 'y':
    with open(path, 'wb') as f:
        pickle.dump(inverted_index, f)
    print("Saved")
else:
    print("Canceled")

In [ ]:
# Load
path = 'Inverted_Index/GTZAN_STFT_II_100_20_CFAR.pkl'
confirmation = input(f"Are you sure you want to load the inverted index table from {path}? (y/n): ")
if confirmation.lower() == 'y':
    with open(path, 'rb') as f:
        inverted_index = pickle.load(f)
    print("Loaded")
else:
    print("Canceled")

### 3.  Query a music from inverted index table

In [ ]:
# Case 1: You do not have the pre-computed inverted indexes
best_match, _, _ = musicToFolderMatching(music_path='Data/mir-2013-GeorgeDataset_snippet(10sec)_1062/',
                                            music_name='blues.00005-snippet-10-20.wav',
                                            folder_path='GTZAN/', inverted_index=None,
                                            window_length_ms=100, hop_length_ms=20, zero_padding=4, delta_T_ms=200,
                                            n_bands=30, target_zone_time_s=0.5, target_zone_time_offset_s=0.1,
                                            target_zone_freq_factor=0.5, progress_bar=True, CFAR_flag=True)

In [ ]:
# Case 2: You have the pre-computed inverted indexes, also there are params can be omitted
best_match, _, _ = musicToFolderMatching(music_path='Data/mir-2013-GeorgeDataset_snippet(10sec)_1062/',
                                            music_name='blues.00011-snippet-10-20.wav',
                                            folder_path='', inverted_index=inverted_index)

### 4.  Match a folder of musics (noisy versions) to the source folder, which contains original musics

In [ ]:
# Case 1: You do not have the pre-computed inverted indexes
predicted_labels, true_labels, inverted_index = folderToFolderMatching(
                                                        folder_path='Data/GTZAN/', inverted_index=None,
                                                        query_folder_path='Data/mir-2013-GeorgeDataset_snippet(10sec)_1062/',
                                                        progress_bar=True, CFAR_flag=True, accuracy_flag=True,
                                                        report_flag=False, confusion_flag=True
                                                        )

In [ ]:
# Case 2: You have the pre-computed inverted indexes
predicted_labels, true_labels, inverted_index = folderToFolderMatching(
                                                        folder_path='', inverted_index=inverted_index,
                                                        query_folder_path='Data/mir-2013-GeorgeDataset_snippet(10sec)_1062/',
                                                        progress_bar=True, CFAR_flag=True, accuracy_flag=True,
                                                        report_flag=False, confusion_flag=True
                                                        )

In [ ]:
# Case 3: You do not yet compute the inverted indexes
# You want to   1. compute the inverted indexes     2. match the music   separately
# Here we can choose if we use mutiThread or not by setting the param `multithread`
# Also we can print the execution time for each step
start_t = time.time()
inverted_index = invertedIndexTable('Data/GTZAN/', multithread=True)
built_ii_t = time.time()
predicted_labels, true_labels, inverted_index = folderToFolderMatching(
                                                        folder_path='', inverted_index=inverted_index,
                                                        query_folder_path='Data/mir-2013-GeorgeDataset_snippet(10sec)_1062/',
                                                        progress_bar=True, CFAR_flag=True, accuracy_flag=True,
                                                        report_flag=False, confusion_flag=True
                                                        )
end_t = time.time()
print(f'Time of Building Inverted Indexes: {(built_ii_t - start_t) :.3f} sec')
print(f'Time of Matching Musics` Fingerprints: {(end_t - built_ii_t):.3f} sec')
print(f'Total Time of Execution: {(end_t - start_t):.3f} sec')